In [352]:
# Step 1: We import all the necessary libraries

import warnings, os, joblib, json, datetime as dt
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.compose      import ColumnTransformer
from sklearn.pipeline     import Pipeline


from sklearn.ensemble import GradientBoostingClassifier as LGBMClassifier


In [378]:
# We define the file paths and configurations here

DATA_PATH      = 'dataset/train.csv'
TIMESTAMP_COL  = 'Timestamp'
FLAG_COL       = 'Monitoring_Alert'
SENSOR_COLS    = ['Surface_Temperature_C']#,'Gearbox_Temperature_C']#,'Motor_Power_kW'
CATEGORICAL_COL = 'Well_Operating_Status'# autodetect if None
TEST_START     = '2023-08-10'            # time-based split , this is time we split our dataset to test against.
RAND_STATE     = 42


In [379]:
# In these steps, we read the data, set the timestamp as index and convert the categorial columns to numerical ones
df = pd.read_csv(DATA_PATH)

df[TIMESTAMP_COL] = pd.to_datetime(df[TIMESTAMP_COL])
df = df.set_index(TIMESTAMP_COL).sort_index()

df = df.loc[:, SENSOR_COLS + [FLAG_COL] + [CATEGORICAL_COL]]

df = pd.get_dummies(df, columns=[CATEGORICAL_COL], prefix='Well_Operating_Status', drop_first=True)


print(df.columns)


Index(['Surface_Temperature_C', 'Monitoring_Alert',
       'Well_Operating_Status_Operating', 'Well_Operating_Status_Standby',
       'Well_Operating_Status_Startup'],
      dtype='object')


### FEATURE ENGINEERING :- We create more features based on rolling averages, slopes and change in slopes

In [380]:
# In this step we resample the sensor data to smoothen it and observe trends. We fill the missing values with interpolation.
sensors = (df[SENSOR_COLS]
      .resample('1H')
      .mean(numeric_only=True)
      .interpolate('time', limit=5))

CATEGORICAL_COL_DUMMIES = [
       'Well_Operating_Status_Operating', 'Well_Operating_Status_Standby',
       'Well_Operating_Status_Startup']

#Here we forward fill the monitoring_alerts based on its last value of 0/1
flags = (
    df[CATEGORICAL_COL_DUMMIES+[FLAG_COL]]
      .resample("1min")
      .ffill()
      .astype(int)
)

df = sensors.join(flags).dropna(subset=[FLAG_COL])
print(f'Loaded {len(df):,} rows, sensors = {SENSOR_COLS}')
print(df.columns)

Loaded 7,007 rows, sensors = ['Surface_Temperature_C']
Index(['Surface_Temperature_C', 'Well_Operating_Status_Operating',
       'Well_Operating_Status_Standby', 'Well_Operating_Status_Startup',
       'Monitoring_Alert'],
      dtype='object')


In [381]:
df.head(10)

,Surface_Temperature_C,Well_Operating_Status_Operating,Well_Operating_Status_Standby,Well_Operating_Status_Startup,Monitoring_Alert
Timestamp,,,,,
2023-01-01 00:00:00,63.397798,1,0,0,0
2023-01-01 01:00:00,62.360361,1,0,0,0
2023-01-01 02:00:00,62.289622,1,0,0,0
2023-01-01 03:00:00,62.735071,1,0,0,0
2023-01-01 04:00:00,63.071355,1,0,0,0
2023-01-01 05:00:00,63.045157,1,0,0,0
2023-01-01 06:00:00,62.371481,1,0,0,0
2023-01-01 07:00:00,61.887134,1,0,0,0
2023-01-01 08:00:00,62.584736,1,0,0,0


In [382]:
import numpy as np

# helper for rolling linear regression slope
def rolling_slope(x: np.ndarray) -> float:
    t = np.arange(len(x))
    # fit a line x ≈ a·t + b, return a
    # np.polyfit is overkill but fine for small windows
    return np.polyfit(t, x, 1)[0]

def make_all_features(
    frame,
    sensor_cols,
    lags=(1, 5, 10),
    rolls=(5,15,24),
):
    X = frame.copy()


    # rolling mean & std (your existing)
    for w in rolls:
        for c in sensor_cols:
            X[f'{c}_mean{w}'] = frame[c].rolling(w).mean()
            X[f'{c}_std{w}']  = frame[c].rolling(w).std()

    # rolling slope & slope change
    for w in rolls:
        slope_col = [f'{c}_slope{w}' for c in sensor_cols]
        X[slope_col] = (
            frame[sensor_cols]
              .rolling(window=w, min_periods=w)
              .apply(rolling_slope, raw=True)
        )
        # second derivative: change of slope
        for c in sensor_cols:
            X[f'{c}_slopechg{w}'] = X[f'{c}_slope{w}'].diff()

    # 5) multi-scale slope differences (short vs long)
    # e.g. slope5 – slope15
    for short, long in [(rolls[0], rolls[1])]:
        for c in sensor_cols:
            X[f'{c}_dslope{short}v{long}'] = (
                X[f'{c}_slope{short}'] - X[f'{c}_slope{long}']
            )

    return X


In [383]:
df_feat = make_all_features(df, SENSOR_COLS)

In [384]:
df_feat.columns

Index(['Surface_Temperature_C', 'Well_Operating_Status_Operating',
       'Well_Operating_Status_Standby', 'Well_Operating_Status_Startup',
       'Monitoring_Alert', 'Surface_Temperature_C_mean5',
       'Surface_Temperature_C_std5', 'Surface_Temperature_C_mean15',
       'Surface_Temperature_C_std15', 'Surface_Temperature_C_mean24',
       'Surface_Temperature_C_std24', 'Surface_Temperature_C_slope5',
       'Surface_Temperature_C_slopechg5', 'Surface_Temperature_C_slope15',
       'Surface_Temperature_C_slopechg15', 'Surface_Temperature_C_slope24',
       'Surface_Temperature_C_slopechg24', 'Surface_Temperature_C_dslope5v15'],
      dtype='object')

In [385]:
df_feat.head()

,Surface_Temperature_C,Well_Operating_Status_Operating,Well_Operating_Status_Standby,Well_Operating_Status_Startup,Monitoring_Alert,Surface_Temperature_C_mean5,Surface_Temperature_C_std5,Surface_Temperature_C_mean15,Surface_Temperature_C_std15,Surface_Temperature_C_mean24,Surface_Temperature_C_std24,Surface_Temperature_C_slope5,Surface_Temperature_C_slopechg5,Surface_Temperature_C_slope15,Surface_Temperature_C_slopechg15,Surface_Temperature_C_slope24,Surface_Temperature_C_slopechg24,Surface_Temperature_C_dslope5v15
Timestamp,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,63.397798,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 01:00:00,62.360361,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 02:00:00,62.289622,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 03:00:00,62.735071,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 04:00:00,63.071355,1,0,0,0,62.770842,0.4703,NaN,NaN,NaN,NaN,-0.027818,NaN,NaN,NaN,NaN,NaN,NaN


In [386]:
X_raw   = df_feat.drop(columns=FLAG_COL)
y_raw   = df[FLAG_COL]

# Drop columsn with all NaN values
X_raw = X_raw.dropna(axis=1, how="all")

# Drop rows with NaNs introduced by lag/roll
mask = X_raw.notna().all(axis=1)
X_raw, y_raw = X_raw[mask], y_raw[mask]

print(X_raw.columns)

Index(['Surface_Temperature_C', 'Well_Operating_Status_Operating',
       'Well_Operating_Status_Standby', 'Well_Operating_Status_Startup',
       'Surface_Temperature_C_mean5', 'Surface_Temperature_C_std5',
       'Surface_Temperature_C_mean15', 'Surface_Temperature_C_std15',
       'Surface_Temperature_C_mean24', 'Surface_Temperature_C_std24',
       'Surface_Temperature_C_slope5', 'Surface_Temperature_C_slopechg5',
       'Surface_Temperature_C_slope15', 'Surface_Temperature_C_slopechg15',
       'Surface_Temperature_C_slope24', 'Surface_Temperature_C_slopechg24',
       'Surface_Temperature_C_dslope5v15'],
      dtype='object')


### MODEL TRAINING , HYPER PARAMETER TUNING, TIME SERIES SPLIT, CROSS VALIDATION

In [388]:
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics         import make_scorer, average_precision_score
import numpy as np

# ----------------------------------------
# 1️⃣ Define the time series split
# ----------------------------------------
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)

# ----------------------------------------
# 2️⃣ Build your pipeline (as before)
# ----------------------------------------
num_cols = list(X_raw.columns)
pre      = ColumnTransformer([('num', StandardScaler(), num_cols)])
clf      = LGBMClassifier(random_state=RAND_STATE)

pipe = Pipeline([
    ('pre',   pre),
    ('model', clf)
])

# ----------------------------------------
# 3️⃣ Set up hyper-parameter grid
# ----------------------------------------
param_dist = {
    'model__n_estimators':   [200, 400, 600, 800],
    'model__learning_rate':  [0.01, 0.03, 0.05, 0.1],
    'model__max_depth':      [3, 5, 7, None],   # None == no limit
    'model__subsample':      [0.6, 0.8, 1.0],
    'model__max_features':   [0.6, 0.8, 1.0],
    # SKLearn-style “regularizers”:
    'model__min_samples_leaf':    [1, 5, 10],
    'model__min_impurity_decrease': [0.0, 1e-4, 1e-3],
}


# ----------------------------------------
# 4️⃣ Choose a scoring metric
# ----------------------------------------
# We care about PR-AUC on the *positive* class
pr_auc_scorer = make_scorer(
    average_precision_score,
    needs_proba=True,
    greater_is_better=True
)

# ----------------------------------------
# 5️⃣ Randomized search with time series CV
# ----------------------------------------
search = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_dist,
    n_iter=30,                  # how many samples from the grid
    scoring=pr_auc_scorer,
    cv=tscv,
    verbose=2,
    n_jobs=-1,
    random_state=RAND_STATE
)

# Fit on your *entire* training span:
split_dt  = pd.to_datetime(TEST_START)
X_train_all = X_raw[X_raw.index < split_dt]
y_train_all = y_raw.loc[X_train_all.index]

search.fit(X_train_all, y_train_all)

print("Best PR-AUC (CV):", search.best_score_.round(4))
print("Best params:", search.best_params_)


Fitting 5 folds for each of 30 candidates, totalling 150 fits


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for 

[CV] END model__learning_rate=0.05, model__max_depth=None, model__max_features=0.8, model__min_impurity_decrease=0.001, model__min_samples_leaf=10, model__n_estimators=400, model__subsample=0.8; total time=   0.5s
[CV] END model__learning_rate=0.05, model__max_depth=5, model__max_features=0.8, model__min_impurity_decrease=0.0001, model__min_samples_leaf=10, model__n_estimators=600, model__subsample=1.0; total time=   0.9s
[CV] END model__learning_rate=0.05, model__max_depth=None, model__max_features=0.8, model__min_impurity_decrease=0.001, model__min_samples_leaf=10, model__n_estimators=400, model__subsample=0.8; total time=   1.2s
[CV] END model__learning_rate=0.05, model__max_depth=5, model__max_features=0.8, model__min_impurity_decrease=0.0001, model__min_samples_leaf=10, model__n_estimators=600, model__subsample=1.0; total time=   1.8s
[CV] END model__learning_rate=0.05, model__max_depth=None, model__max_features=0.8, model__min_impurity_decrease=0.001, model__min_samples_leaf=10, 

#### EVALUATING THE MODEL

In [389]:
# ----------------------------------------
# 6️⃣ Evaluate on hold-out test set
# ----------------------------------------
X_test = X_raw[X_raw.index >= split_dt]
y_test = y_raw.loc[X_test.index]

best_pipe = search.best_estimator_
y_prob    = best_pipe.predict_proba(X_test)[:,1]
y_pred    = (y_prob > 0.8).astype(int)   # or tune this threshold separately

from sklearn.metrics import classification_report
print("\nTest set report:")
print(classification_report(y_test, y_pred, digits=3))


Test set report:
              precision    recall  f1-score   support

           0      0.977     0.987     0.982      1551
           1      0.853     0.763     0.806       152

    accuracy                          0.967      1703
   macro avg      0.915     0.875     0.894      1703
weighted avg      0.966     0.967     0.966      1703



In [390]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_sensors(df, height_per_sensor=250):
    """
    Plot each column of df in its own subplot (shared x-axis).

    Parameters
    ----------
    df : pandas.DataFrame
        DateTimeIndex + one numeric column per sensor.
    height_per_sensor : int, optional
        Pixel height allotted to each subplot (default 250).

    Returns
    -------
    fig : plotly.graph_objects.Figure
    """
    n = len(df.columns)                     # <— works for any “n sensors”
    fig = make_subplots(
        rows=n,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.015              # tighter spacing
    )

    for i, col in enumerate(df.columns, start=1):
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=df[col],
                name=col,
                line_width=1
            ),
            row=i,
            col=1
        )
        fig.update_yaxes(title_text=col, row=i, col=1)

    fig.update_layout(
        height=max(400, n * height_per_sensor),   # scale canvas height
        showlegend=False,
        hovermode="x unified",
        margin=dict(t=30, b=30, l=60, r=20)
    )
    return fig

# ------- USAGE -------
plot_df = X_test.copy()
plot_df['Monitoring_Health_Status']=y_test
plot_df['Monitoring_Health_Status_pred']=y_pred
#plot_df = plot_df.set_index('Timestamp').sort_index()


fig = plot_sensors(plot_df)      # df = your cleaned DataFrame
fig.show()


#### EXPORTING THE TRAINED MODEL PIPELINE

In [391]:
# --------------------------------------------------------------
# STEP 8 – EXPORT PIPELINE (for real-time inferencing)
# --------------------------------------------------------------
os.makedirs('models', exist_ok=True)
joblib.dump(search.best_estimator_, "models/gbm_pipeline.joblib")


['models/gbm_pipeline.joblib']